<a href="https://colab.research.google.com/github/RoshanChongder/IMAGE-PROCESSING-AND-FACIAL-EMOTION-DETECTION/blob/master/modelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Mount Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing required Modules 

In [ ]:
import os
import sys

import pandas as pd
import numpy as np


# keras is free source deep learning library in python
# from this library different layer will be imported
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


Downloading the csv file from the mounted google drive 

Importing the data set and pre-viewing  

In [ ]:
# Method to read CSV Files
def read_CSV( path ):
    try:
        file = pd.read_csv( path )
        return file
    except FileNotFoundError:
        print( "CSV File not found at " + path )
        return None
    except Exception:
        print(" Unknown error appeared ")
        return None

# reading the csv file 

data_set = read_CSV('drive/MyDrive/EmotionDetection/fer2013/fer2013.csv') 
print( data_set.info() ) # checking info of the data set 
print( data_set.head())  # showing the first ew line of the data set 


Now , as the data set is loaded , we will just extract the data from the csv file and tehn load then in  train , ytrain , xtest and ytest . 


In [ ]:
# Addition of data into list from CSV for training and public testing
def data_Addition() :
    global x_train , y_train , x_test , y_test , data_set
    d = {3:0} 
    for row_count,row in data_set.iterrows():
        #if row['emotion'] in [1,2] or ( d[3]>5800 and row['emotion']==3 and row['Usage'] == 'Training' ) :  continue 
        
        if row['emotion'] in [1,2] or ( row['emotion'] == 3  and d[ row['emotion'] ] > 4000  and row['Usage'] == 'Training' ) : continue 
        value = row['pixels'].split(' ')   # extracting the pixels as a list
        try :
            if 'Training' in row['Usage'] :        # if the current column is for Training
                x_train.append(np.array(value,'float32'))        # adding the pixels in the x axis
                y_train.append(row['emotion'])

                if row['emotion'] in d :
                    d[ row['emotion'] ]+=1
                else :
                    d[ row['emotion'] ] = 1 

                #{3: 5801, 0: 3995, 4: 4830, 6: 4965, 5: 3171}
                # added this to restrict some of the happy data - ( d[3]>5800 and row['emotion']==3 and row['Usage'] == 'Training' ) 

            # adding emotion in the y axis
            # elif 'PublicTest' in row['Usage']:    # if the current column is for testing
            #     x_test.append(np.array( value,'float32'))
            #     y_test.append( row['emotion'])

                # {0: 467, 4: 653, 6: 607, 3: 895, 5: 415}
            else : 
                # also including the private test set  over here 
                x_test.append(np.array( value,'float32'))
                y_test.append( row['emotion']) 

                # {0: 958, 4: 1247, 6: 1233, 3: 1774, 5: 831}  # Testing :  6043 

        except:
            print(" Error occurred at row number " + row_count)
            print("Data Set in that row is " + row )

    print(d)

# Now we will do training and public testing

x_train , y_train = [] , []   # data the will be used for training will added in this two lists
x_test , y_test = [] , []     # data that will be used for public testing will be added here

data_Addition()    # addition of data in the lists for training and testing

print( "Training : " , len(x_train) , "Testing : " , len(x_test) )

# checking the lists
print( x_train[:2])
print( y_train[:2])



In [ ]:
# copy the prev cell  ******** Aritra *********
# Addition of data into list from CSV for training and public testing
def data_Addition() :
    global x_train , y_train , x_test , y_test , data_set
    d = {3:0} 
    for row_count,row in data_set.iterrows():
        if row['emotion'] in [1,2] or ( row['emotion'] == 3  and d[ row['emotion'] ] > 4000  and row['Usage'] == 'Training' ) : continue 
        value = row['pixels'].split(' ')   # extracting the pixels as a list
        try :
            if 'Training' in row['Usage'] :                      # if the current column is for Training

                x_train.append(np.array(value,'float32'))        # adding the pixels in x axis
                y_train.append(row['emotion'])                   # adding the actual emotion tag in y axis 

                if row['emotion'] in d :
                    d[ row['emotion'] ]+=1
                else :
                    d[ row['emotion'] ] = 1 
            else : 

                x_test.append(np.array( value,'float32'))
                y_test.append( row['emotion'])  

        except:
            print(" Error occurred at row number " + row_count)
            print("Data Set in that row is " + row )

    print(d)

# Now we will do training and public testing

x_train , y_train = [] , []   # data the will be used for training will added in this two lists
x_test , y_test = [] , []     # data that will be used for public testing will be added here

data_Addition()    # addition of data in the lists for training and testing

print( "Training : " , len(x_train) , "Testing : " , len(x_test) )


Don't execute the next as it is the copy of the previous code cell 

In [ ]:
# Addition of data into list from CSV for training and public testing
'''
def data_Addition() :
    global x_train , y_train , x_test , y_test , data_set
    d = dict()

    for row_count,row in data_set.iterrows():
        if row['emotion'] in [1,2] :  continue 
        value = row['pixels'].split(' ')   # extracting the pixels as a list
        try :
            if 'Training' in row['Usage'] :        # if the current column is for Training
                x_train.append(np.array(value,'float32'))        # adding the pixels in the x axis
                y_train.append(row['emotion'])
                
                if row['emotion'] in d :
                    d[ row['emotion'] ]+=1
                else :
                    d[ row['emotion'] ] = 1 
            
            # adding emotion in the y axis
            elif 'PublicTest' in row['Usage']:    # if the current column is for testing
                x_test.append(np.array( value,'float32'))
                y_test.append( row['emotion'])

        except:
            print(" Error occurred at row number " + row_count)
            print("Data Set in that row is " + row )

    print(d )

# Now we will do training and public testing

x_train , y_train = [] , []   # data the will be used for training will added in this two lists
x_test , y_test = [] , []     # data that will be used for public testing will be added here

data_Addition()    # addition of data in the lists for training and testing

# checking the lists
#print( x_train[:2])
#print( y_train[:2])

'''

As the keras module only takes numpy array as input parameters , we need to convert the lists into numpy arrays .

In [ ]:
# Method to Convert from list to Numpy Arrays
def Convert_to_np_Array():
    global x_train, x_test, y_test , y_train
    # Converting list to numpy Array
    x_train = np.array(x_train, 'float32')
    y_train = np.array(y_train, 'float32')
    x_test = np.array(x_test, 'float32')
    y_test = np.array(y_test, 'float32')



# As the Keras Module only takes numpy arrays as input
# we need to convert this lists into numpy arrays

Convert_to_np_Array()
print( type(x_train) )  # checking if the type has changed to numpy or not 


Resaclling of x_train and y_train and then reshaping them into one d array 


In [ ]:
def Rescale():
    # Normalizing the data
    # why data normalization is required - https://www.import.io/post/what-is-data-normalization-and-why-is-it-important/
    # how it's work - https://www.educative.io/edpresso/data-normalization-in-python
    # read Out - https://www.mathsisfun.com/data/standard-deviation.html

    global x_train , x_test , y_test , y_train

    # we are basically rescaling
    x_train -= np.mean(x_train, axis=0)
    x_train /= np.std(x_train, axis=0)  # CENTRALIZING THE DATA

    x_test -= np.mean(x_test, axis=0)
    x_test /= np.std(x_test, axis=0)


def Reshape( width , height ):

    global x_train , y_train , x_test , y_test

    x_train = x_train.reshape(x_train.shape[0], width, height, 1)
    x_test = x_test.reshape(x_test.shape[0], width, height, 1)
    # WHAT THIS FUNCTION DOES TO_CATEGORICAL
    y_train = np_utils.to_categorical(y_train,num_classes=7)
    y_test = np_utils.to_categorical(y_test,num_classes=7)
    
    print( ' Reshape method is called  ')

# Rescalling the data
Rescale()

# Reshaping the x train and y train in to a one d array
Reshape(48,48)


Now the main comes - designing the CNN 

In [ ]:

def Design_CNN():

    # The number of epochs is a hyperparameter
    # that defines the number times that the learning algorithm will work through the entire training dataset
    features = 64
    Batch_size = 64
    Label = 7
    epoch = 100
    global  x_train, y_train
    model = Sequential()

    ## Layer 1
    # adding layers
    # Conv2d is used as the image are in 2d format
    # here we are trying extract input
    # Relu is a rectifier

    # Search Kernal size
    model.add(Conv2D(features,kernel_size=(3,3),activation='relu',input_shape=(x_train.shape[1:])))
    model.add(Conv2D(features,kernel_size=(3, 3),activation='relu'))

    # adding a max pooling 2D layer
    # It mainly helps to control over fitting
    # can use average pooling layer also
    model.add( MaxPool2D(pool_size=(2,2),strides=(2,2)) )

    # adding a drop out layer
    model.add(Dropout(0.3))

    ## 2ND layer
    model.add(Conv2D(features, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.4))

    ## 3RD Layer
    model.add(Conv2D(2*features, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(2*features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    model.add( Flatten() )

    # adding dense layers
    model.add(Dense(2**3 * features, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2 ** 3 * features, activation='relu'))
    model.add(Dropout(0.2))

    # Adding the final layers
    model.add(Dense(Label,activation='softmax')) # Activation is softmax as we want to bind in the 7 labels of 0ptions

    model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
    model.fit(x_train,y_train,batch_size=Batch_size,epochs=epoch,verbose=1,validation_data=(x_test,y_test), shuffle=True )

    # Saving the model

    EmotionDetectJson = model.to_json()
    with open("fer.json","w") as file :
        file.write(EmotionDetectJson)
    model.save_weights("fer.h5")




# Start designing Our CNN

# To build the model we will be using sequential Type

Design_CNN()


